In [1]:
import numpy as np
import scipy
import qiskit
import qiskit.quantum_info as qi

import sys
sys.path.append('./src')


from lchs import *
from lcu import *
from lchs_bos import *
from utils_synth import *


In [65]:
def bos_prolong_select_oracle(theta_array, num_qubits_per_qumode, lcu_circ:qiskit.QuantumCircuit, debug:bool=False) -> qiskit.QuantumCircuit:
    ## See (7.55) in https://arxiv.org/pdf/2201.08309
    num_terms = len(theta_array)
    num_qubits_control = nearest_num_qubit(num_terms)
    num_qubits_op = num_qubits_per_qumode
    bin_string = '{0:0'+str(num_qubits_control)+'b}'
    ##
    # qbr = qiskit.QuantumRegister(num_qubits_control)
    # qbr_ancilla = qiskit.QuantumRegister(1)
    # qcr = qiskit.ClassicalRegister(1)
    qbr, qbr_ancilla, qhamr = lcu_circ.qregs
    qcr = lcu_circ.cregs[0]
    # qmr = c2qa.QumodeRegister(num_qumodes=1, num_qubits_per_qumode=num_qubits_per_qumode)
    # select_circ = c2qa.CVCircuit(qbr, qbr_ancilla, qmr, qcr)
    # select_circ = qiskit.QuantumCircuit(qbr, qbr_ancilla, qcr)
    for i in range(num_terms):
        ibin = bin_string.format(i)[::-1] ## NOTE: Qiskit uses reverse order
        ## For 0-control
        for q in range(len(ibin)):
            qbit = ibin[q]
            if qbit == '0':
                lcu_circ.x(q)
        ## multiple-control X targeting ancilla qubit
        lcu_circ.mcx(qbr, qbr_ancilla[0])
        # Sanwitch the the controlled-U bosonic gate
        try:
            # select_circ.cv_c_r(theta_array[i], qmr[0], qbr_ancilla[0])
            lcu_circ.cp(theta_array[i], qbr_ancilla[0], qhamr[0])
        except:
            print("Error in bos_gate")
            print("theta_array[i] =", theta_array[i])
            print("num_qubits_per_qumode =", num_qubits_per_qumode)
            print("num_qubits_control =", num_qubits_control)
            raise
        ## reset to undo the computation
        ## For 0-control
        for q in range(len(ibin)):
            qbit = ibin[q]
            if qbit == '0':
                lcu_circ.x(q)
        lcu_circ.h(qbr_ancilla[0])
        lcu_circ.measure(qbr_ancilla[0], qcr[0])
        ## if else uncompute
        with lcu_circ.if_test((qcr[0], 1)) as else_:
            lcu_circ.h(qbr[num_qubits_control-1])
            lcu_circ.mcx(qbr[:num_qubits_control-1], qbr[num_qubits_control-1])
            lcu_circ.h(qbr[num_qubits_control-1])
            lcu_circ.x(qbr_ancilla[0]) ## reset the ancilla qubit if measure 1
    print(lcu_circ.qregs)


def bos_prolong_lcu(coeff_array:numpy.array, theta_array:numpy.array, num_qubits_per_qumode, initial_state_circ=None,verbose:int=0, qiskit_api:bool=False, debug:bool=False) -> qiskit.QuantumCircuit:
    num_terms = len(coeff_array) #len(absorbed_unitaries)
    num_qubits_control = nearest_num_qubit(num_terms)
    num_qubits_op = num_qubits_per_qumode
    ## separate the real and imaginary parts
    real_nonneg, real_neg, imag_nonneg, imag_neg = split_complex_array(coeff_array)
    norm_real_nonneg = numpy.sum(real_nonneg)
    # norm_real_neg = numpy.sum(real_neg)
    # norm_imag_nonneg = numpy.sum(imag_nonneg)
    # norm_imag_neg = numpy.sum(imag_neg)
    print("LCU bos debug")
    recon_coeff = (real_nonneg-real_neg)+1j*(imag_nonneg-imag_neg)
    print("  Coeff Reconstruction error =", numpy.linalg.norm(recon_coeff-coeff_array,ord=2))

    ## prep circuit for real and imag parts
    real_nonneg_prep_circ = prep_oracle(real_nonneg, qiskit_api=qiskit_api)
    # real_neg_prep_circ = prep_oracle(real_neg, qiskit_api=qiskit_api)
    # imag_nonneg_prep_circ = prep_oracle(imag_nonneg, qiskit_api=qiskit_api)
    # imag_neg_prep_circ = prep_oracle(imag_neg, qiskit_api=qiskit_api)
    ## select circuit for real and imag parts
    

    ## prolonged LCU circuit
    ## use 1 extra ancilla qubit to take the result of multiple-control, 
    ## sanwitch the single-control bosonic gate with multiple-control qubit gate
    qbr = qiskit.QuantumRegister(num_qubits_control)
    qbr_ancilla = qiskit.QuantumRegister(1)
    qcr = qiskit.ClassicalRegister(1)
    qhamr = qiskit.QuantumRegister(num_qubits_op)
    lcu_anc_cr = qiskit.ClassicalRegister(num_qubits_control)
    # qmr = c2qa.QumodeRegister(num_qumodes=1, num_qubits_per_qumode=num_qubits_per_qumode)
    ## Real Non-negative part
    # lcu_circ_real_nonneg = c2qa.CVCircuit(qbr, qbr_ancilla, qmr, qcr)
    lcu_circ_real_nonneg = qiskit.QuantumCircuit(qbr, qbr_ancilla, qhamr, qcr)
    if initial_state_circ:
        lcu_circ_real_nonneg.append(initial_state_circ.copy(), list(range(num_qubits_control+num_qubits_op+1))[num_qubits_control:])
    ## Apply the preparation oracle
    lcu_circ_real_nonneg.append(real_nonneg_prep_circ, list(range(num_qubits_control)))
    ## Apply the selection oracle
    print(lcu_circ_real_nonneg.qregs)
    bos_prolong_select_oracle(theta_array, num_qubits_per_qumode, lcu_circ_real_nonneg, debug=debug)
    ## Apply the preparation oracle dagger
    lcu_circ_real_nonneg.append(real_nonneg_prep_circ.inverse(), list(range(num_qubits_control)))
    # lcu_circ_real_nonneg.measure(qbr, lcu_anc_cr) ## measure the LCU controls
    lcu_circ_real_nonneg.save_statevector()
    # return lcu_circ_real_nonneg.reverse_bits(), norm_real_nonneg  ## NOTE: i.e., not in qiskit order after reverse_bits
    return lcu_circ_real_nonneg

In [67]:
coeff_array = np.array([1,2,3])
theta_array = np.array([0.1,0.2,0.3])
num_qubits_per_qumode = 1


anc_lcu_circ = bos_prolong_lcu(coeff_array, theta_array, num_qubits_per_qumode, qiskit_api=True)
anc_lcu_circ.draw()


LCU bos debug
  Coeff Reconstruction error = 0.0
[QuantumRegister(2, 'q27'), QuantumRegister(1, 'q28'), QuantumRegister(1, 'q29')]
[QuantumRegister(2, 'q27'), QuantumRegister(1, 'q28'), QuantumRegister(1, 'q29')]


┌───────────────────────────────────────────────┐┌───┐       ┌───┐  »
q27_0: ┤0                                              ├┤ X ├──■────┤ X ├──»
       │  State Preparation(0.40825,0.57735,0.70711,0) │├───┤  │    ├───┤  »
q27_1: ┤1                                              ├┤ X ├──■────┤ X ├──»
       └───────────────────────────────────────────────┘└───┘┌─┴─┐  └───┘  »
  q28: ──────────────────────────────────────────────────────┤ X ├─■───────»
                                                             └───┘ │P(0.1) »
  q29: ────────────────────────────────────────────────────────────■───────»
                                                                           »
c23: 1/════════════════════════════════════════════════════════════════════»
                                                                           »
«                  ┌──────                   ───────┐                         »
«q27_0: ───────────┤       ─────────■───────        ├────────■────────────────»
«                  │         ┌───┐┌─┴─┐┌───┐        │ ┌───┐  │    ┌───┐       »
«q27_1: ───────────┤ If-0  ──┤ H ├┤ X ├┤ H ├  End-0 ├─┤ X ├──■────┤ X ├───────»
«       ┌───┐┌─┐   │         ├───┤└───┘└───┘        │ └───┘┌─┴─┐  └───┘  ┌───┐»
«  q28: ┤ H ├┤M├───┤       ──┤ X ├──────────        ├──────┤ X ├─■───────┤ H ├»
«       └───┘└╥┘   └──╥───   └───┘           ───────┘      └───┘ │P(0.2) └───┘»
«  q29: ──────╫───────╫──────────────────────────────────────────■────────────»
«             ║ ┌─────╨─────┐                                                 »
«c23: 1/══════╩═╡ c23_0=0x1 ╞═════════════════════════════════════════════════»
«             0 └───────────┘                                                 »
«             ┌──────                   ───────┐ ┌───┐       ┌───┐          »
«q27_0: ──────┤       ─────────■───────        ├─┤ X ├──■────┤ X ├──────────»
«             │         ┌───┐┌─┴─┐┌───┐        │ └───┘  │    └───┘          »
«q27_1: ──────┤ If-0  ──┤ H ├┤ X ├┤ H ├  End-0 ├────────■───────────────────»
«       ┌─┐   │         ├───┤└───┘└───┘        │      ┌─┴─┐         ┌───┐┌─┐»
«  q28: ┤M├───┤       ──┤ X ├──────────        ├──────┤ X ├─■───────┤ H ├┤M├»
«       └╥┘   └──╥───   └───┘           ───────┘      └───┘ │P(0.3) └───┘└╥┘»
«  q29: ─╫───────╫──────────────────────────────────────────■─────────────╫─»
«        ║ ┌─────╨─────┐                                                  ║ »
«c23: 1/═╩═╡ c23_0=0x1 ╞══════════════════════════════════════════════════╩═»
«        0 └───────────┘                                                  0 »
«          ┌──────                   ───────┐ »
«q27_0: ───┤       ─────────■───────        ├─»
«          │         ┌───┐┌─┴─┐┌───┐        │ »
«q27_1: ───┤ If-0  ──┤ H ├┤ X ├┤ H ├  End-0 ├─»
«          │         ├───┤└───┘└───┘        │ »
«  q28: ───┤       ──┤ X ├──────────        ├─»
«          └──╥───   └───┘           ───────┘ »
«  q29: ──────╫───────────────────────────────»
«       ┌─────╨─────┐                         »
«c23: 1/╡ c23_0=0x1 ╞═════════════════════════»
«       └───────────┘                         »
«       ┌──────────────────────────────────────────────────┐ statevector 
«q27_0: ┤0                                                 ├──────░──────
«       │  State Preparation Dg(0.40825,0.57735,0.70711,0) │      ░      
«q27_1: ┤1                                                 ├──────░──────
«       └──────────────────────────────────────────────────┘      ░      
«  q28: ──────────────────────────────────────────────────────────░──────
«                                                                 ░      
«  q29: ──────────────────────────────────────────────────────────░──────
«                                                                 ░      
«c23: 1/═════════════════════════════════════════════════════════════════
«

In [70]:
from qiskit_aer import AerSimulator
from qiskit import transpile
sim = AerSimulator()
result = sim.run(anc_lcu_circ.decompose(reps=3), shots=1).result()
counts = result.get_counts(anc_lcu_circ)
print(counts)

result.get_statevector(anc_lcu_circ)

{'0': 1}
Statevector([ 3.33333333e-01-4.02752977e-16j,
             -6.05139802e-16-4.71404521e-01j,
              6.66666667e-01-8.76025592e-16j,
              3.89252089e-16+4.71404521e-01j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j,
              0.00000000e+00+0.00000000e+00j],
            dims=(2, 2, 2, 2))


In [44]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator

qubits = QuantumRegister(2)
clbits = ClassicalRegister(3)
circuit = QuantumCircuit(qubits, clbits)
(q0, q1) = qubits
(c0, c1,c2) = clbits
 
circuit.h(q0)
circuit.cx(q0, q1)
circuit.measure(q0, c0)




with circuit.if_test((c0, 1)) as else_:
    circuit.x(q1)
circuit.measure(q1, c1)
circuit.z(q1)
circuit.measure(q1, c2)

circuit.save_statevector()

circuit.draw()

┌───┐     ┌─┐                                        statevector 
q19_0: ┤ H ├──■──┤M├─────────────────────────────────────────────░──────
       └───┘┌─┴─┐└╥┘   ┌──────   ┌───┐ ───────┐ ┌─┐┌───┐┌─┐      ░      
q19_1: ─────┤ X ├─╫────┤ If-0  ──┤ X ├  End-0 ├─┤M├┤ Z ├┤M├──────░──────
            └───┘ ║    └──╥───   └───┘ ───────┘ └╥┘└───┘└╥┘      ░      
                  ║ ┌─────╨─────┐                ║       ║              
c16: 3/═══════════╩═╡ c16_0=0x1 ╞════════════════╩═══════╩══════════════
                  0 └───────────┘                1       2

In [45]:
from qiskit_aer import AerSimulator
sim = AerSimulator()
result = sim.run(circuit, shots=100).result()
counts = result.get_counts(circuit)
print(counts)

result.get_statevector(circuit)

{'000': 45, '001': 55}
Statevector([ 1.+0.j,  0.+0.j, -0.+0.j, -0.+0.j],
            dims=(2, 2))


In [56]:
AerSimulator()._cached_basis_gates

['ccx',
 'ccz',
 'cp',
 'crx',
 'cry',
 'crz',
 'cswap',
 'csx',
 'cu',
 'cu1',
 'cu2',
 'cu3',
 'cx',
 'cy',
 'cz',
 'diagonal',
 'ecr',
 'h',
 'id',
 'mcp',
 'mcphase',
 'mcr',
 'mcrx',
 'mcry',
 'mcrz',
 'mcswap',
 'mcsx',
 'mcu',
 'mcu1',
 'mcu2',
 'mcu3',
 'mcx',
 'mcx_gray',
 'mcy',
 'mcz',
 'multiplexer',
 'p',
 'pauli',
 'r',
 'roerror',
 'rx',
 'rxx',
 'ry',
 'ryy',
 'rz',
 'rzx',
 'rzz',
 's',
 'sdg',
 'store',
 'swap',
 'sx',
 'sxdg',
 't',
 'tdg',
 'u',
 'u1',
 'u2',
 'u3',
 'unitary',
 'x',
 'y',
 'z']